In [1]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import uuid
import os
import urllib
import json
from json import JSONEncoder
from uuid import UUID

global Postcode
Postcode = "BT3"

class Scraper:
    def __init__(self, url: str = 'https://www.propertypal.com') -> None:
        self.driver = Chrome(ChromeDriverManager().install())
        self.driver.get(url)
        
    def ivan_accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        
        try:
            time.sleep(1)
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))

            button = self.driver.find_element(By.XPATH, xpath)
            button.click()
        except TimeoutException:
            print("No Cookies Found")

    def accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        button = self.driver.find_element(By.XPATH, xpath)
        button.click()

    
    def search_word(self, xpath, pc = Postcode):

        search = self.driver.find_element(By.XPATH, xpath)
        self.driver.implicitly_wait(10)
        ActionChains(self.driver).move_to_element(search).click(search).perform()
        ActionChains(self.driver).send_keys(pc).perform()

    def scroll_down(self):
        
        self.driver.execute_script("window.scrollTo(0, 540)") 
    
    def search_rent(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[2]')
        rent.click()

    def search_sale(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[1]')
        rent.click()

    def button_click(self,xpath):
        button = self.driver.find_element_by_xpath(xpath)
        button.click()
    
    def find_container(self, xpath: str = '//*[@id="body"]/div[3]/div/div[1]/div/ul'):
        self.container = self.driver.find_element(By.XPATH, xpath)
        return self.container

##
# NAVIGATE THE WEBSITE
##

if __name__ == "__main__":
    bot = Scraper()
    bot.ivan_accept_cookies()
    time.sleep(2)
    bot.button_click('//a[@href="/login"]')
    time.sleep(2)
    bot.search_word('//input[@placeholder="Email address"]','sopranotony233@gmail.com')
    time.sleep(2)
    bot.search_word('//input[@placeholder="Password"]','sopranotony321')
    time.sleep(2)
    bot.button_click('//*[@id="root"]/div/div/div/div/div/div[1]/div/div[2]/div[2]/form/button')
    time.sleep(2)
    bot.button_click('//a[@class="mainnav-logo"]')  
    time.sleep(2)
    bot.search_word('//*[@id="searchForm"]/div/div[1]')
    time.sleep(2)
    bot.search_rent()
    time.sleep(2)
    

    #CREATE THE LIST OF LINKS
    list_links = []
    print("Finding elements...")
    while True:
        container = bot.find_container()
        items = container.find_elements(By.XPATH, './li')
        for i in items:
            try:
                house = i.find_element(By.XPATH, './/a[2]')
                link = house.get_attribute('href')
                list_links.append(link)
            except:
                print("no href found")
        
        try:
            bot.button_click('//a[@class="btn paging-next"]')
        except NoSuchElementException:
            print("end of list")
            break

    #GRAB INFO FROM EACH LINK AND STORE
    print("Grabbing info...")
    prop_dict = {"fr-id": [],
            "id": [],
            "Link": [],
             "Summary": [],
            "Address": [],
            "Price": [],
            "Image links": []
            }
    for link in list_links:
        im = link[-6:]
        im2 = link[28:35]
        prop_dict["fr-id"].append(im2+im)
        id = uuid.uuid4()
        prop_dict["id"].append(id)
        bot.driver.get(link)
        prop_dict["Link"].append(link)
        time.sleep(0.5)
        summary = bot.driver.find_element(By.XPATH, '//div[@class="prop-heading-brief"]')
        prop_dict["Summary"].append(summary.text)
        time.sleep(0.5)
        info = bot.driver.find_element(By.XPATH, '//div[@class="prop-summary-row"]')
        address = info.find_element(By.XPATH, './/h1')
        prop_dict["Address"].append(address.text)
        time.sleep(0.5)
        price = bot.driver.find_element(By.XPATH, '//div[@class="prop-price"]')
        prop_dict["Price"].append(price.text)
        img_list = bot.driver.find_elements(By.XPATH, '//div[@class="Slideshow-slides SlideshowCarousel"]//img')
        img_links = []

        
        for img in img_list:
            try:
                link = img.get_attribute('src')
                img_links.append(link)
            except:
                print('No src found')
        prop_dict["Image links"].append(img_links)


    os.mkdir(f"/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/Property_Photos/{Postcode}")
    image_directory = os.path.dirname(f"/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/Property_Photos/{Postcode}/")
    img_link_ct= -1
    for img_list in prop_dict["Image links"]:
        img_link_ct += 1
        img_ct = 0
        for url in img_list:
            try:
                img_ct += 1
                req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                with open(f"{image_directory}/{prop_dict['fr-id'][img_link_ct]}_{str(img_ct)}.jpg", "wb") as f:
                    with urllib.request.urlopen(req) as r:
                        f.write(r.read())
            except IndexError:
                pass 

    old_default = JSONEncoder.default

    def new_default(self, obj):
        if isinstance(obj, UUID):
            return str(obj)
        return old_default(self, obj)

    JSONEncoder.default = new_default
 
    with open(f'/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/raw_data/{Postcode}.json', 'w') as f:
        json.dump(prop_dict, f, indent = 4)

    df = pd.DataFrame(prop_dict)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.134/chromedriver] found in cache
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_59493/1180928362.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_59493/1180928362.py:64: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.driver.find_element_by_xpath(xpath)
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_59493/1180928362.py:56: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  rent = self.driver.find_element_by_xpath('//*[@id="

Finding elements...
no href found
no href found
end of list
Grabbing info...


In [2]:
df

,fr-id,id,Link,Summary,Address,Price,Image links
0,50-quay769630,cc0ffd24-44d7-4ba8-95f4-49ff34ba45d5,https://www.propertypal.com/50-quaygate-19-sta...,1 Bed Apartment To Rent,"50 QUAYGATE, 19 STATION STREET,\nBelfast BT3 9DA",£695\n/ month + fees,[https://media.propertypal.com/sd/p/768430/326...
1,the-arc769505,e0259162-e736-41f3-9e3c-0174d29a22e9,https://www.propertypal.com/the-arc-226-2b-que...,2 Bed Apartment To Rent,"THE ARC, 2.26 2B QUEENS ROAD,\nTitanic Quarter...","£1,100\n/ month",[https://media.propertypal.com/sd/p/768305/325...
2,57-quay769418,79dbce7f-0cf8-4b1f-acc5-0e1e0f851e51,https://www.propertypal.com/57-quay-gate-stati...,2 Bed Apartment To Rent,"57 QUAY GATE,\nStation Road, Belfast BT3 9DA",£800\n/ month,[https://media.propertypal.com/sd/p/768218/325...
3,918-bil768716,f439676b-795e-43e0-92b7-50abafdcb805,https://www.propertypal.com/918-bills-included...,1 Bed Apartment To Rent,"9.18 BILLS INCLUDED,\nThe Arc, Queens Road, Be...","£1,100\n/ month + fees",[https://media.propertypal.com/sd/p/767516/325...
4,4-18-th767245,e4f20348-1464-4086-9819-0ba9f386841e,https://www.propertypal.com/4-18-the-arc-titan...,2 Bed Apartment To Rent,"4-18 THE ARC, TITANIC QUARTER, 2C QUEENS ROAD,...","£1,250\n/ month + fees",[https://media.propertypal.com/sd/p/766045/323...
5,apt-34-765126,b2afaf4c-9c2c-431d-9897-bd6e0eea827f,https://www.propertypal.com/apt-34-quay-gate-s...,3 Bed Apartment To Rent,"APT 34, QUAY GATE,\nStation Street, Belfast BT...",£925\n/ month + fees,[https://media.propertypal.com/sd/p/763926/322...
6,the-arc762119,fa2780cc-de42-4fa8-a722-27335f872e16,https://www.propertypal.com/the-arc-titanic-qu...,2 Bed Apartment To Rent,"THE ARC, TITANIC QUARTER, BELFAST,\nBelfast BT...","£1,550\n/ month + fees",[https://media.propertypal.com/sd/p/760919/319...


In [4]:
bt4 = [
        "15-holy769842",
        "14-sand769820",
        "80-belm769451",
        "a9-alto769347",
        "9-campb515070",
        "85-uppe769189",
        "3-castl769168",
        "apt-14-768940",
        "27-garn768910",
        "57-vict768648",
        "apt2-4-768598",
        "28-park768346",
        "apartme768044",
        "12-inve768013",
        "68-belm356006",
        "19-asto767775",
        "127-cas767763",
        "apt-9-3767651",
        "9-pims-765427",
        "apt-3-g765358",
        "53-uppe765329",
        "40-norw765201",
        "apartme764434",
        "79-uppe730603",
        "18-stra762633",
        "85-uppe606480",
        "apt-4-4676830",
        "apt-a-5746507"]

In [5]:
len(bt4)

28